# Language Model

1. Implemented N-gram language modeling.
2. Implemented RNN language modeling.

In [1]:
# !pip install numpy scikit-learn tqdm matplotlib
# !pip install -U spacy
# !python -m spacy download en_core_web_sm

### Load Data

In [2]:
train_texts = []
with open('./data/train.txt', 'r') as fp:
    for line in fp:
        train_texts.append(line)
        
valid_texts = []
with open('./data/valid.txt', 'r') as fp:
    for line in fp:
        valid_texts.append(line)
        
test_texts = []
with open('./data/input.txt', 'r') as fp:
    for line in fp:
        test_texts.append(line)


In [3]:
train_texts[0]

' aer banknote berlitz calloway centrust cluett fromstein gitano guterman hydro-quebec ipo kia memotec mlx nahb punts rake regatta rubens sim snack-food ssangyong swapo wachter \n'

### Preprocessing

In [256]:
import re
import string

class Preprocesser(object):
    def __init__(self, punctuation=True, url=True, number=True):
        self.punctuation = punctuation
        self.url = url
        self.number = number
    
    def apply(self, text):
        text = self._lowercase(text)
        if self.url:
            text = self._remove_url(text)
        if self.punctuation:
            text = self._remove_punctuation(text)
        if self.number:
            text = self._remove_number(text)
        text = re.sub(r'\s+', ' ', text)
        return text

    def _remove_punctuation(self, text):
        text = text.translate(str.maketrans('','', string.punctuation))
        return text
    
    def _remove_url(self, text):
        text = re.sub(r'\S*https?:\S*','',text)
        return text
    
    def _remove_number(self, text):
        text = text.translate(str.maketrans('','', string.digits))
        return text
    
    def _lowercase(self, text):
        text = text.lower()
        return text

### Tokenization

In [257]:
import spacy
nlp = spacy.load('en_core_web_sm')

def tokenize(text):

    doc = nlp(text)
    tokens = [token.text for token in doc]
    
    return tokens
    

## 1. N-gram
1. Implement N-gram (Bigram).
2. Implement Good Turing smoothing.
3. Implement Kneser-Ney smoothing.

### 1.1 Implement a bigram for language modeling

In [258]:
from collections import defaultdict
from tqdm.notebook import tqdm

class BiGram(object):
    
    def __init__(self):
        ''' Construction function of BiGram.
            Params:
                uni_count: a dictionary with default value 0
                bi_count: a dictionary that each value is a dictionary with default value 0
        '''

        self.uni_count = defaultdict(lambda: 0)
        self.bi_count = defaultdict(lambda: defaultdict(lambda: 0))
        
        
    def fit(self, texts):
        self._unigram_count(texts)
        self._bigram_count(texts)
        
    
    def _unigram_count(self, texts):
        ''' Count tokens, and store in self.uni_count
            Input
                texts: a list of text
        '''

        for text in tqdm(texts,desc='unigram count'):
            clean_text = preprocesser.apply(text)
            tokens = tokenize(clean_text)
            self.uni_count.update({token:self.uni_count[token]+tokens.count(token) for token in tokens})

    
    def _bigram_count(self, texts):
        ''' Count tokens in bigram way, and store in self.bi_count
            Input
                texts: a list of text
        '''

        for text in tqdm(texts,desc='bigram count'):
            clean_text = preprocesser.apply(text)
            tokens = tokenize(clean_text)

            for i,token in enumerate(tokens):
                if i<len(tokens)-1:
                    self.bi_count[token][tokens[i+1]]+=1

    
    def probability(self, w1, w2):
        ''' Given two tokens, calculate the bigram probability
            Input
                w1: the first token of bigram
                w2: the second token of bigram
        '''
        prob = 0.
        
        # Laplace Smoothing with reconstituted counts c* = [c(w1,w2)+1]*c(w1)/(c(w1)+V)

        prob = ((self.bi_count[w1][w2]+1)/(self.uni_count[w1]+len(self.uni_count.values())))*self.uni_count[w1]

        return prob
    
    
    def predict(self, w):
        w_next = None

        max_p = 0
        for word in list(self.bi_count[w].keys()):
            p = self.probability(w,word)
            if p>max_p:
                max_p=p
                w_next=word

        return w_next

### 1.2 Implement Good Turing smoothing 

In [259]:
from collections import Counter
from scipy.optimize import curve_fit

# PowerLaw funtion
def func_powerlaw(x, m, c, c0):
    return c0 + x**m * c

In [303]:
class GoodTuring(object):
    
    def __init__(self, bigram):
        ''' Construction function of Good Turing.
            Input
                bigram: Bigram model
            Params:
                uni_count: a dictionary with default value 0
                bi_count: a dictionary that each value is a dictionary with default value 0
                -----------------
                For bigram
                bi_nc: a dictionary with default value 0, the count of things we've seen c times.
                bi_c_star: (c+1)*N_c+1 / N_c, page 64 of slides (lecture 5).
                bi_N: \sum c*N_c, page 64 of slides (lecture 5).
                
                For unigram
                uni_nc: a dictionary with default value 0, the count of things we've seen c times.
                uni_c_star: (c+1)*N_c+1 / N_c, page 64 of slides (lecture 5).
                uni_N: \sum c*N_c, page 64 of slides (lecture 5).
            
        '''
        self.uni_count = bigram.uni_count
        self.bi_count = bigram.bi_count
        
        self.uni_nc = defaultdict(lambda: 0)
        self.bi_nc = defaultdict(lambda: 0)
        
        self.uni_c_star = defaultdict(lambda: 0)
        self.bi_c_star = defaultdict(lambda: 0)
        
        self.uni_N = 0
        self.bi_N = 0
        
        
    def fit(self, texts):
        self._calc_N_c()
        self._calc_c_star_and_N()
        
    
    def _calc_N_c(self):
        ''' Count the frequency of frequency c, and store to self.nc.
        '''
 
        # update uni_nc
        k = list(Counter(self.uni_count.values()).keys())
        v = list(Counter(self.uni_count.values()).values())
        self.uni_nc.update({k[i]:v[i] for i in range(len(k))})
        
        # update bi_nc
        bi_dicts = list(self.bi_count.values())
        for d in bi_dicts:
            k = list(Counter(d.values()).keys())
            v = list(Counter(d.values()).values())
            self.bi_nc.update({k[i]:self.bi_nc[k[i]]+v[i] for i in range(len(k))})


        
        # fit emperical Nk with best fit power law for c>100
        cc,cov = curve_fit(func_powerlaw,list(self.uni_nc.keys()),list(self.uni_nc.values()),maxfev = 5000)
        for c in range(101,max(list(self.uni_nc.keys()))):
            nc = func_powerlaw(c,cc[0],cc[1],cc[2])
            if nc>1:
                self.uni_nc[c]=nc
            else:
                self.uni_nc[c]=1            

        
        
    def _calc_c_star_and_N(self):
        ''' Calculate c_star and N. 
        '''
        


        for c in list(self.uni_nc.keys()):
            self.uni_c_star[c] = (c+1)*(self.uni_nc[c+1]/self.uni_nc[c])
            
        self.uni_N = sum(self.uni_count.values())

        for c in list(self.bi_nc.keys()):
            self.bi_c_star[c] = (c+1)*(self.bi_nc[c+1]/self.bi_nc[c])
        
        self.bi_N = sum([sum(x.values()) for x in list(self.bi_count.values())])


        
    

    def probability(self, w1, w2):
        ''' Given two words, calculate the GT probability
                p_GT = c_star / N, if c != 0
                p_GT = N_1 / N, if c = 0
                
                p = p_GT(w1, w2) / p_GT(w1)
                
            Input
                w1: the first word
                w2: the second word
                
        '''
        prob = 0.
        
        uni_c = self.uni_count[w1]
        
        bi_c = self.bi_count[w1][w2]
        if uni_c == 0:
            # P_GT_uni = self.uni_nc[1] / self.uni_N
            # approximating p = 1E-5 for new words as N1(self.uni_N) is very large
            P_GT_uni = 1E-5
        else:
            if self.uni_c_star[uni_c]==0:
                # Naive approach to approximate c* by subracting c by 0.75
                P_GT_uni = (uni_c-0.75)/self.bi_N
            else:
                P_GT_uni = self.uni_c_star[uni_c] / self.uni_N
            
        
        if bi_c == 0:
            # P_GT_w12 = self.bi_nc[1] / self.bi_N
            P_GT_bi = 1E-5
        else:
            if self.bi_c_star[bi_c]==0:
                P_GT_bi = (bi_c - 0.75) / self.bi_N
            else:
                P_GT_bi = self.bi_c_star[bi_c] / self.bi_N

        
        prob = P_GT_bi / P_GT_uni
        
        return prob

    
    def predict(self, w):
        ''' Given a word, find a word with the highest probability
            Input
                w: a word
        '''
        
        w_next = None
        
        max_p = 0
        for word in list(self.bi_count[w].keys()):
            p = self.probability(w,word)
            if p>=max_p:
                max_p=p
                w_next=word

        
        return w_next

### 1.3 Implement Kneser-Ney smoothing

In [304]:
class KneserNey(object):
    
    def __init__(self, bigram, d=0.75):
        ''' Construction function of KneserNey.
            Params:
                uni_count: a dictionary with default value 0
                bi_count: a dictionary that each value is a dictionary with default value 0
            
        '''
        
        self.uni_count = bigram.uni_count
        self.bi_count = bigram.bi_count
        
        self.num_bigram_types = 0
        self.novel_continuation = defaultdict(lambda: 0)
        self.novel_previous = defaultdict(lambda: 0)
        self.p_continuation = defaultdict(lambda: 0)
        self.lam = defaultdict(lambda: 0)
        
        self.d = d
        
    
    def fit(self, texts):
        self._calc_num_bigram_types()
        self._calc_novel_continuation_and_novel_previous()
        self._calc_P_continuation()
        self._calc_lambda()
        
    
    def _calc_num_bigram_types(self):
        ''' Calculate the number of bigram types, and store in self.num_bigram_types
            
        '''
                
        self.num_bigram_types = sum([len(x)-sum([1 for v in x.values() if v==0]) for x in list(self.bi_count.values())])
                  
    
    def _calc_novel_continuation_and_novel_previous(self):
        ''' Calculate novel continuation, and novel previous, 
            and store them in self.novel_continuation and self.novel_previous
        '''
        
        self.novel_continuation.update({w:sum([1 for x in list(self.bi_count.values()) if not x[w]==0]) for w in list(self.uni_count.keys())})
        self.novel_previous.update({w:len(self.bi_count[w]) for w in list(self.bi_count.keys())})
    
    
    def _calc_P_continuation(self):
        ''' Calculate p continuation, and store in self.p_continuation.
        '''
        
        self.p_continuation.update({w:self.novel_continuation[w]/self.num_bigram_types for w in list(self.novel_continuation.keys())})
    
    def _calc_lambda(self):
        ''' Calculate lambda, and store in self.lam.

        '''
        
        self.lam.update({w:(self.d/self.uni_count[w])*self.novel_previous[w] for w in list(self.bi_count.keys()) if not self.uni_count[w]==0})
        
        
    def probability(self, w1, w2):
        ''' Given two words, calculate the KN probability
                
            Input
                w1: the first word
                w2: the second word
        '''
        
        prob = 0.
        
        
        if not self.uni_count[w1]==0:
            prob = (max(self.bi_count[w1][w2] - self.d,0)/self.uni_count[w1]) + self.lam[w1]*self.p_continuation[w2]
        else:
            prob = 1E-5
                    
        return prob
    
    
    def predict(self, w):
        ''' Given a word, find a word with the highest probability
            Input
                w: a word
        '''
        
        pred = None
        
        max_p = 0
        for word in list(self.bi_count[w].keys()):
            p = self.probability(w,word)
            if p>max_p:
                max_p=p
                pred=word
                
        return pred
                

### 1.4 Implement Perplexity 
**Hint:** Multiplication of probabilities may lead to an overflow problem. One trick is to move the computation to the logarithm space. Therefore, you could use summation instead of multiplication to calculate perplexity.

In [305]:
import math

def perplexity(model, texts):
    ''' Calculate the perplexity score.
        Inputs
            model: the model you want to evaluate (BiGram, GoodTuring, or KneserNey)
            texts: a list of validation text
        Output
            perp: the perplexity of the model on texts
    '''
    perp = 1.
    
    bigrams = []
    logsum=0

    for text in texts:
        tokens = tokenize(preprocesser.apply(text))
        for i,token in enumerate(tokens):
            if i < (len(tokens) - 1):
                bigram = (token,tokens[i+1])
                if bigram not in bigrams:
                    bigrams.append(bigram)
                    p = model.probability(bigram[0],bigram[1])
                    logsum += math.log(p)
    
    perp = math.exp((-1/len(bigrams))*logsum)

    
    return perp


### 1.5 Calculate the perplexity of three models

Run the following cell to obtian the perplexity of BiGram, Good Turing, and Kneser-Ney.


In [263]:
# Train Bigram
bigram = BiGram()
bigram.fit(train_texts)

# Perplexity
bigram_perplexity = perplexity(bigram, valid_texts)
print(f'The perplexity of Bigram is: {bigram_perplexity:.4f}')

unigram count:   0%|          | 0/42068 [00:00<?, ?it/s]

bigram count:   0%|          | 0/42068 [00:00<?, ?it/s]

The perplexity of Bigram is: 8.3391


In [306]:
# Train Good Turing
gt = GoodTuring(bigram)
gt.fit(train_texts)

# Perplexity
gt_perplexity = perplexity(gt, valid_texts)
print(f'The perplexity of Good Turing is: {gt_perplexity:.4f}')

The perplexity of Good Turing is: 92.2569


In [307]:
# For Kneser-Ney
kn = KneserNey(bigram, d=0.75)
kn.fit(train_texts)

# Perplexity
kn_perplexity = perplexity(kn, valid_texts)
print(f'The perplexity of Kneser-Ney is: {kn_perplexity:.4f}')

The perplexity of Kneser-Ney is: 68.3211


### 1.6 Use N-gram model make predictions


#### 1.6.1 Bigram

In [218]:
import random

sampled_texts = random.sample(test_texts, 30)
for i, text in enumerate(sampled_texts):
    clean_text = preprocesser.apply(text)
    tokens = tokenize(clean_text)
    pred = bigram.predict(tokens[-1])
    print(f'{i} ==> {text.strip()}, prediction: {pred}')

0 ==> japanese stocks dropped early monday but by late morning were ___, prediction: nt
1 ==> high-grade unsecured notes sold through dealers by major corporations in ___, prediction: the
2 ==> under the agreement gulf must give the u.s. government N ___, prediction: n
3 ==> the issues of companies with ties to the junk bond ___, prediction: prices
4 ==> ncnb for example has been one of the highest rate ___, prediction: of
5 ==> just when it seemed safe to go back into stocks ___, prediction: in
6 ==> on the other hand it 's not necessarily a good ___, prediction: news
7 ==> though dow has aggressively diversified into specialty chemicals and pharmaceuticals ___, prediction: and
8 ==> i ca n't imagine that you 'll talk to anyone ___, prediction: who
9 ==> the strike by customs officers tax collectors treasury workers and ___, prediction: the
10 ==> american <unk> co. offering of N common shares via merrill ___, prediction: lynch
11 ==> the fed 's efforts at <unk> were partly <unk> sund

#### 1.6.2 Good Turing

In [232]:
sampled_texts = random.sample(test_texts, 30)
for i, text in enumerate(sampled_texts):
    clean_text = preprocesser.apply(text)
    tokens = tokenize(clean_text)
    pred = gt.predict(tokens[-1])
    print(f'{i} ==> {text.strip()}, prediction: {pred}')

0 ==> one merc broker compared the action in the s&p pit ___, prediction: unilab
1 ==> the auto maker 's decision to let word of the ___, prediction: company
2 ==> in recent years u.s. steelmakers have supplied about N N ___, prediction: n
3 ==> and forecasts project a N N to N N growth ___, prediction: in
4 ==> the office of thrift supervision has been <unk> centrust to ___, prediction: n
5 ==> mr. spielvogel is n't part of the board nor are ___, prediction: the
6 ==> japan 's wholesale prices in september rose N N from ___, prediction: n
7 ==> first came his predictable <unk> he charged the coalition of ___, prediction: the
8 ==> i 'm not going to worry about one day 's ___, prediction: n
9 ==> big board chairman john j. phelan said yesterday the circuit ___, prediction: breakers
10 ==> friday capped a week of steadily rising crude oil prices ___, prediction: for
11 ==> south africa freed the anc 's sisulu and seven other ___, prediction: major
12 ==> mr. <unk> said he estimated that

#### 1.6.3 Kneser-Ney

In [230]:
sampled_texts = random.sample(test_texts, 30)
for i, text in enumerate(sampled_texts):
    clean_text = preprocesser.apply(text)
    tokens = tokenize(clean_text)
    pred = kn.predict(tokens[-1])
    print(f'{i} ==> {text.strip()}, prediction: {pred}')

0 ==> i 've had a lot of people trying to sell ___, prediction: its
1 ==> <unk> newsletter writer francis <unk> <unk> actually did it he ___, prediction: said
2 ==> i 'd like to see that initiative and i have ___, prediction: been
3 ==> ideal said it expects to complete the transaction early next ___, prediction: year
4 ==> generally overcapacity in commercial real estate is dropping from its ___, prediction: n
5 ==> the land to be purchased by the joint venture has ___, prediction: been
6 ==> refiners say margins picked up in september and many industry ___, prediction: s
7 ==> mr. <unk> also said the fda has asked <unk> pharmaceutical ___, prediction: concern
8 ==> also in july ohio newspapers disclosed $ N in corporate ___, prediction: bonds
9 ==> mr. <unk> says he has paid more than $ N ___, prediction: n
10 ==> moreover to complete the entire <unk> glass purchase and <unk> ___, prediction: the
11 ==> nevertheless tandem faces a variety of challenges the biggest being ___, predicti

## 2. RNN

1. Initialize parameters
2. Prepare Data
3. Implement the model
4. Train your model
5. Evaluate your model


### 2.1 Initialize parameters for the model

In [328]:

learning_rate = 1E-6
batch_size = 1
hidden_size = 30
embedding_size = 50
num_epochs = 20
window_size = 16

### 2.2 Data preparation

1. Build a vocabulary.
2. Prepare the training data.
3. Prepare the validation data.

In [320]:
import numpy as np
import torch
import torch.nn as nn
from gensim.models import Word2Vec
from tqdm import tqdm

device = torch.device('cuda')

toks = []
vocab = []
txt = train_texts[:11000]

for sen in tqdm(txt):
    toks.append(tokenize(Preprocesser().apply(sen[1:])))
    for word in tokenize(Preprocesser().apply(sen[1:])):
        vocab.append(word)
        
vocab = np.unique(np.array(vocab))
embeds = Word2Vec(sentences=toks, vector_size=embedding_size, window=window_size, min_count=1, workers=4)
embeds.train(vocab, epochs=50, total_examples=1)
vocab_vectors = embeds.wv
print(vocab_vectors['the'].shape)

toks = []
txt = train_texts[:10000]
for sen in tqdm(txt):
    toks.append(tokenize(Preprocesser().apply(sen[1:])))
    
data_train = toks

toks = []
txt = train_texts[10000:11000]
for sen in tqdm(txt):
    toks.append(tokenize(Preprocesser().apply(sen[1:])))
    
data_val = toks

print(len(vocab), len(data_train), len(data_val))



100%|██████████| 11000/11000 [03:20<00:00, 54.78it/s]


(50,)


100%|██████████| 1000/1000 [00:09<00:00, 107.25it/s]

8810 10000 1000


### 2.3 Build your model

1. Create a model.
2. Add an embedding layer as the first layer.
3. Add a RNN cell (GRU or LSTM) as the next layer.
4. Add a output layer.
5. Given a sequence words, for each word, predict the next word.

In [321]:
class Embed():
    
    def __init__(self):
        
        self.vectorize = vocab_vectors
        
    def transform(self, data):
            
        y = self.vectorize[data]
            
        y = np.expand_dims(y, axis=0)
        
        y = np.expand_dims(y, axis=0)
        
        y = torch.from_numpy(y)
        
        return y

class my_RNN(nn.Module):
    
    def __init__(self, input_size, hidden_size):
        
        super().__init__()
        
        self.embed = Embed()
        self.rnn_cell = nn.LSTM(input_size = input_size, hidden_size = hidden_size, batch_first=True)
        self.out = nn.Linear(hidden_size, input_size)
        
    def forward(self, x):
        
        y = self.embed.transform(x) #shape -> (#batches,#sequence, 200)
        #print(y.shape)
        y = self.rnn_cell(y)
        #print(y[0].shape)
        y = self.out(y[0]) #shape -> (200)
        
        return y



### 2.4 Setup the training step and train the model

In [325]:

model = my_RNN(embedding_size, hidden_size)

model.train()

optimizer = torch.optim.Adam(model.parameters(), lr = learning_rate)

loss = nn.MSELoss()

for ep in range(num_epochs):
    
    loss_list = []
    
    for sen in data_train:
        
        for i in range(len(sen) - 1):
    
            optimizer.zero_grad(set_to_none=True)

            y = model.forward(sen[i])

            L = loss(y, model.embed.transform(sen[i + 1]))

            L.backward()

            optimizer.step()

            loss_list.append(float(L))
            
    print(f'epoch {ep} Loss :', sum(loss_list) / len(loss_list))

model.eval()
           

epoch 0 Loss : 0.3068762429843218
epoch 1 Loss : 0.2271545296609093
epoch 2 Loss : 0.21613400710957675
epoch 3 Loss : 0.2099400190585789
epoch 4 Loss : 0.20637583929712178
epoch 5 Loss : 0.2042634355741353
epoch 6 Loss : 0.20294589873288282
epoch 7 Loss : 0.20206749277950983
epoch 8 Loss : 0.20144154673101053
epoch 9 Loss : 0.20096965117803628
epoch 10 Loss : 0.20059776913500033
epoch 11 Loss : 0.20029436644595855
epoch 12 Loss : 0.20003983952893054
epoch 13 Loss : 0.19982134757369496
epoch 14 Loss : 0.1996301017144724
epoch 15 Loss : 0.1994598968441271
epoch 16 Loss : 0.19930622047674168
epoch 17 Loss : 0.19916572027893972
epoch 18 Loss : 0.19903585109353997
epoch 19 Loss : 0.19891466216375114


my_RNN(
  (rnn_cell): LSTM(50, 30, batch_first=True)
  (out): Linear(in_features=30, out_features=50, bias=True)
)

### 2.5 Evaluate the model by Calculating the model's perplexity on the valid set.

In [326]:
import math

loss = nn.MSELoss()

perp = 0.


loss_list = []

pred_len = 0
    
for sen in data_val:
        
    for i in range(len(sen) - 1):
    
        optimizer.zero_grad(set_to_none=True)

        y = model.forward(sen[i])

        L = loss(y, model.embed.transform(sen[i + 1]))

        loss_list.append(float(L))
        
        pred_len += 1
            
loss = sum(loss_list) / len(loss_list)

perp = math.exp(loss)


print(f'The perplexity of of RNN based model is: {perp:.4f}')


The perplexity of of RNN based model is: 1.2137


### 2.6 Use RNN language modeling make predictions 


In [327]:

import random

sampled_texts = random.sample(test_texts, 30)
for i, text in enumerate(sampled_texts):
    clean_text = preprocesser.apply(text)
    tokens = tokenize(clean_text)
    pred = model.forward(tokens[-1]).to('cpu').detach().numpy().squeeze()
    pred = vocab_vectors.most_similar(positive=[pred], topn=10)[0][0]
    print(f'{i} ==> {text.strip()}, prediction: {pred}')



0 ==> many of the nation 's <unk> executives <unk> friday 's ___, prediction: control
1 ==> two years ago about the only point of comparison was ___, prediction: almost
2 ==> in a <unk> <unk> therefore the region 's real-estate market ___, prediction: production
3 ==> indeed <unk> and other leading hong kong toy makers were ___, prediction: shareholders
4 ==> i am happy to see the spirit of the people ___, prediction: making
5 ==> robert <unk> and <unk> <unk> professors of finance at the ___, prediction: includes
6 ==> most people he says have no idea what a massacre ___, prediction: against
7 ==> that system is designed to separate <unk> program trades from ___, prediction: par
8 ==> <unk> portfolio manager james craig was n't <unk> when friday ___, prediction: costs
9 ==> some analysts say uncertainty about washington 's anti-takeover policy was ___, prediction: almost
10 ==> there 's also a host of new programs trying to ___, prediction: shareholders
11 ==> in these locations mr. <u